In [1]:
import os
import sys

sys.path.append("../code")
from util import extract_pdf_to_txt

In [2]:
done = os.listdir("../text")
for filename in os.listdir("../pdf"):
    if (filename + ".txt") in done:
        continue
    extract_pdf_to_txt(filename, filename, "local")

In [3]:
done = os.listdir("../text")
len(done)

116

# Google Sheets


In [132]:
import gspread
import time
import math

In [6]:
# connect to the service account
gc = gspread.service_account(filename="../creds.json")

sh = gc.open("Paper Extraction Evaluation").sheet1
# sh.update_cell(10, 3, "helloooo")

# RegEx


In [7]:
import re

import sys

sys.path.append("../code")

from util import extract_doi_to_txt, get_string_from_text_file

In [117]:
# ***Last line should not contain "|"***
regex_method = r"""
    ^\s*\d*[.]?\s?(((O(\s)?ur)|P(\s)?roposed)(\s))?M(\s)?ethod(s)?(ology)?\s*$|# "method" "methods" "methodology"
    ^\s*\d*[.]?\s?(M(\s)?aterial(s)?|B(\s)?ackground)((\s)and(\s)Method(s)?)?\s*$| # "materials and methods" "material" "materials"
    ^\s*\d*[.]?\s?(P(\s)?roposed(\s))?A(\s)?pproach(es)?\s*$| # "approach" "proposed approach"
    ^\s*\d*[.]?\s?(O(\s)?ur(\s))?m(\s)?odel(s)?\s*$| # "our model" "model"
    ^\s*\d*[.]?\s?T(\s)?raining\s*$ # "training"
    
"""

# ***Last line should not contain "|"***
regex_result_only = r"""
    ^\s*\d*[.]?\s?R(\s)?esult(s)?((\s)and(\s)(discussion|analysis))?\s*$| # "result" "results" "results and discussion"
    ^\s*\d*[.]?\s?(P(\s)?erformance(\s))?A(\s)?nalysis\s*$| # "performance analysis"
    ^\s*\d*[.]?\s?((E(\s)?mpirical|E(\s)?xperimental)(\s))?Result(s)?\s*$| # "empirical result" "experimental results"
    ^\s*\d*[.]?\s?(E(\s)?xperiment|D(\s)?ownstream(\s)task)(s)?(\s)and(\s)Result(s)?\s*$ # "experiment and results"
    
"""
# ***Last line should not contain "|"***
regex_result_others = r"""
    ^\s*\d*[.]?\s?E(\s)?xperiment(s)?((\s)and(\s)(discussion|application)(s)?)?\s*$| # "experiment" "experiments" "experiments and discussions"
    ^\s*\d*[.]?\s?E(\s)?valuation(s)?((\s)and(\s)comparison(s)?)?\s*$| # "evaluation" "evaluations" "evaluation and comparisons"
    ^\s*\d*[.]?\s?C(\s)?omparison(s)?((\s)((with(\s)State-of-the-arts)|(to(\s)Human(\s)Performance)))?\s*$| # "comparison" "comparison with state-of-the-arts"
    ^\s*\d*[.]?\s?A(\s)?pplication(s)?\s*$ # "application"
    
"""

# ***Last line should not contain "|"***
regex_extra = r"""
    ^\s*\d*[.]?\s?(F(\s)?urther(\s))?D(\s)?iscussion(s)?\s*$| # "discussion" "discussions"
    ^\s*\d*[.]?\s?R(\s)?eference(s)?\s*$| # "reference" "references"
    ^\s*\d*[.]?\s?(Discussion(s)?(\s)and(\s))?C(\s)?onclusion(s)?\s*$| # "conclusion"
    ^\s*\d*[.]?\s?L(\s)?iterature(\s)Review\s*$| # "literature review"
    ^\s*\d*[.]?\s?I(\s)?ntroduction((\s)and(\s)Motivating(\s)Work)?\s*$| # "introduction"
    ^\s*\d*[.]?\s?A(\s)?bstract\s*$| # "abstract"
    ^\s*\d*[.]?\s?L(\s)?imitation(s)?((\s)(and|&)(\s)(Discussion(s)|(Societal(\s)Impact)|(Future(\s)Work(s)?))?)?\s*$| # "limitation" "limitations"
    ^\s*\d*[.]?\s?A(\s)?ppendi(x)?(ces)?\s*$| # "appendix" "appendices"
    ^\s*\d*[.]?\s?B(\s)?ibliography\s*$| # "bibliography"
    ^\s*\d*[.]?\s?A(\s)?cknowledgement(s)?\s*$| # "acknowledgement" "acknowledgements"
    ^\s*\d*[.]?\s?T(\s)?ables(\s)and(\s)Figures\s*$| # "tables and figures"
    ^\s*\d*[.]?\s?(R(\s)?elated|P(\s)?revious)(\s)W(\s)?ork(s)?\s*$| # "related work"
    ^\s*\d*[.]?\s?A(\s)?nalysis\s*$| # "analysis"
    ^\s*\d*[.]?\s?(A(\s)?dditional(\s))?A(\s)?blation(\s)S(\s)?tud(y|ies)\s*$| # "analysis" "ablation study" "additional ablation study"
    ^\s*\d*[.]?\s?P(\s)?reliminar(y|ies)\s*$| # "preliminary"
    ^\s*\d*[.]?\s?E(\s)?xperiment(\s)Setup\s*$| # "experiment setup"
    ^\s*\d*[.]?\s?B(\s)?ackground\s*$| # "background"
    ^\s*\d*[.]?\s?(B(\s)?roader(\s))?I(\s)?mpact(s)?\s*$| # "broader impact"
    ^\s*\d*[.]?\s?(C(\s)?hallenges(\s)and(\s))?F(\s)?uture(\s)Direction(s)?\s*$| # "challenges and future direction"
    ^\s*\d*[.]?\s?A(\s)?cknowledgment(s)?\s*$| # "Acknowledgments"
    ^\s*\d*[.]?\s?(E(\s)?xperimental(\s))?S(\s)?etup\s*$| # "experimental setup"
    ^\s*\d*[.]?\s?O(\s)?verview\s*$| # "overview"
    ^\s*\d*[.]?\s?(S(\s)?afety(\s))?C(\s)?onsideration(s)?\s*$| # "Safety Considerations"
    ^\s*\d*[.]?\s?F(\s)?uture(\s)work(s)?\s*$| # "future work"
    ^\s*\d*[.]?\s?R(\s)?esource(s)?\s*$ # "resources"
    
    
    
"""

regex_total = (
    regex_method
    + "|"
    + regex_result_only
    + "|"
    + regex_result_others
    + "|"
    + regex_extra
)

# Testing


In [76]:
# custom_regex = "hi"
custom_regex = None
if custom_regex:
    regex_total += "\n    "
    if custom_regex[0] != "|":
        custom_regex = "|" + custom_regex
    regex_total += custom_regex
print(regex_total)

# add custom rules
regex_total_cp = re.compile(regex_total, re.IGNORECASE | re.VERBOSE | re.MULTILINE)
regex_method_cp = re.compile(regex_method, re.IGNORECASE | re.VERBOSE)
regex_result_others_cp = re.compile(regex_result_others, re.IGNORECASE | re.VERBOSE)
regex_result_only_cp = re.compile(regex_result_only, re.IGNORECASE | re.VERBOSE)


    ^\d?[.]?\s?Method(s)?(ology)?$|# "method" "methods" "methodology"
    ^\d?[.]?\s?Material(s)?((\s)and(\s)Methods)?$ # "materials and methods" "material" "materials"
|
    ^\d?[.]?\s?Result(s)?((\s)and(\s)discussion)?$| # "result" "results" "results and discussion"
    ^\d?[.]?\s?Empirical(\s)Result(s)?$ # "result" "results" "results and discussion"
    
|
    ^\d?[.]?\s?Experiment(s)?$| # "experiment" "experiments"
    ^\d?[.]?\s?Evaluation(s)?((\s)and(\s)comparison(s)?)?$ # "evaluation" "evaluations" "evaluation and comparisons"
|
    ^\d?[.]?\s?Discussion(s)?$| # "discussion" "discussions"
    ^\d?[.]?\s?Reference(s)?$| # "reference" "references"
    ^\d?[.]?\s?Conclusion$| # "conclusion"
    ^\d?[.]?\s?Literature(\s)Review$| # "literature review"
    ^\d?[.]?\s?Introduction$| # "introduction"
    ^\d?[.]?\s?Abstract$| # "abstract"
    ^\d?[.]?\s?Limitation(s)?$| # "limitation" "limitations"
    ^\d?[.]?\s?Appendi(x)?(ces)?$| # "appendix" "appendices"
    ^\d?[.]?\s?Bibliography

In [77]:
text = get_string_from_text_file(
    filename="Arjovsky et al. - 2017 - Wasserstein GAN.pdf"
)

method_idx = -1
result_idx = -1
result_flag = False  # because "Results" section can be called many names
section_start_point_list = []
for i, match in enumerate(regex_total_cp.finditer(text)):
    section = match.group()
    if regex_method_cp.search(section):
        method_idx = i

    # If there is an exact match with "Results"
    if regex_result_only_cp.search(section):  # if there is an exact match
        result_idx = i
        result_flag = True
    # If not, and if exact match has not been found yet, look for other names
    if regex_result_others_cp.search(section) and not result_flag:
        result_idx = i

    section_start_point_list.append(match.start())
    print(match)
    print([match.group()])
    # break


print(method_idx)
print(result_idx)
print(section_start_point_list)

reading data from Arjovsky et al. - 2017 - Wasserstein GAN.pdf.txt...
string successfully retrieved.

<re.Match object; span=(140, 154), match='1 Introduction'>
['1 Introduction']
<re.Match object; span=(18649, 18668), match='4 Empirical Results'>
['4 Empirical Results']
<re.Match object; span=(26878, 26892), match='5 Related Work'>
['5 Related Work']
<re.Match object; span=(34696, 34706), match='References'>
['References']
-1
1
[140, 18649, 26878, 34696]


In [101]:
list_length = len(section_start_point_list)
start_idx = section_start_point_list[method_idx]
end_idx = (
    -1 if method_idx >= list_length - 1 else section_start_point_list[method_idx + 1]
)
method_text = text[start_idx:end_idx] if method_idx != -1 else ""
end_idx = None if result_idx >= list_length - 1 else result_idx + 1

start_idx = section_start_point_list[result_idx]
end_idx = (
    -1 if result_idx >= list_length - 1 else section_start_point_list[result_idx + 1]
)
result_text = text[start_idx:end_idx] if result_idx != -1 else ""

# Function Definition


In [17]:
def extract(
    filename: str,
    regex_base: str,
    custom_regex: str | None = None,
    row_idx: int | None = -1,
) -> None:
    print("\n====================================================")
    regex_total = regex_base
    if custom_regex:
        regex_total += "\n    "
        if custom_regex[0] != "|":
            custom_regex = "|" + custom_regex
        regex_total += custom_regex

    # print("Final RegEx:\n", regex_total)

    # add custom rules
    regex_total_cp = re.compile(
        regex_total,
        re.IGNORECASE | re.VERBOSE | re.MULTILINE,
    )
    regex_method_cp = re.compile(regex_method, re.IGNORECASE | re.VERBOSE)
    regex_result_others_cp = re.compile(regex_result_others, re.IGNORECASE | re.VERBOSE)
    regex_result_only_cp = re.compile(regex_result_only, re.IGNORECASE | re.VERBOSE)

    text = get_string_from_text_file(filename)

    method_idx = -1
    result_idx = -1
    result_flag = False  # because "Results" section can be called many names
    section_start_point_list = []
    extracted_section_name_list = []
    for i, match in enumerate(regex_total_cp.finditer(text)):
        section = match.group()
        if regex_method_cp.search(section):
            method_idx = i

        # If there is an exact match with "Results"
        if regex_result_only_cp.search(section):  # if there is an exact match
            result_idx = i
            result_flag = True
        # If not, and if exact match has not been found yet, look for other names
        if regex_result_others_cp.search(section) and not result_flag:
            result_idx = i

        section_start_point_list.append(match.start())
        # print(match)
        extracted_section_name_list.append(match.group().strip())
        print([match.group()])

    list_length = len(section_start_point_list)
    if list_length == 0:
        print("No sections found...")
        return
    start_idx = section_start_point_list[method_idx]
    end_idx = (
        len(text)
        if method_idx >= list_length - 1
        else section_start_point_list[method_idx + 1]
    )
    end_idx = min(end_idx, start_idx + 40000)
    method_text = text[start_idx:end_idx] if method_idx != -1 else ""

    start_idx = section_start_point_list[result_idx]
    end_idx = (
        len(text)
        if result_idx >= list_length - 1
        else section_start_point_list[result_idx + 1]
    )
    # 40000 -> char limit per section (50000 char limit on gs api)
    end_idx = min(end_idx, start_idx + 40000)
    result_text = text[start_idx:end_idx] if result_idx != -1 else ""

    if row_idx != -1:
        # connect to the service account
        gc = gspread.service_account(filename="../creds.json")

        sh = gc.open("Paper Extraction Evaluation").sheet1
        print(f"\nUpdating google sheet row {row_idx}")
        sh.update_cell(row_idx, 1, filename[:-4])
        sh.update_cell(row_idx, 2, "\n".join(extracted_section_name_list))
        sh.update_cell(row_idx, 4, method_text)
        sh.update_cell(row_idx, 7, result_text)
    else:
        print("############ METHOD TEXT:\n", method_text)
        print("############ RESULT TEXT:\n", result_text)
        print(
            "############ Extracted Section List:\n",
            "\n".join(extracted_section_name_list),
        )

# Loop through text files


In [137]:
math.floor(math.log10(100)) + 2

4

In [139]:
# row number that has already been executed
breakpoint_row_num = 88
with open("../evaluation_paper_list.txt", "r") as paper_list:
    papers = paper_list.read().splitlines()

for i, file in enumerate(papers, 3):
    if i <= breakpoint_row_num:
        continue
    if (i - 3) >= 100:
        break
    flag = False
    filename = file[math.floor(math.log10(i)) + 2 :]
    while not flag:
        try:
            extract(filename, regex_total, row_idx=i)
            flag = True
        except gspread.exceptions.APIError as e:
            print("API error. Retrying...")
            time.sleep(3)
        except:
            print("Error.")
            break
    if not flag:
        break


reading data from Yao et al. - 2023 - Tree of Thoughts Deliberate Problem Solving with .pdf.txt...
string successfully retrieved.

['Abstract']
['1 Introduction']
['2 Background']
['4 Experiments']
['5 Related Work']
['6 Discussion']
['9Broader Impact']
['Acknowledgements']
['References']

Updating google sheet row 89

reading data from Kavasidis et al. - 2017 - Brain2Image Converting Brain Signals into .pdf.txt...
string successfully retrieved.

['ABSTRACT']
['1 INTRODUCTION']
['2 RELATED WORK']
['3 METHOD']
['4 PERFORMANCE ANALYSIS']
['ACKNOWLEDGMENTS']
['REFERENCES']
Error.


# Individual File


In [118]:
extract(
    "He et al. - Thinking Image Color Aesthetics Assessment Models.pdf",
    regex_total,
    row_idx=67,
)


reading data from He et al. - Thinking Image Color Aesthetics Assessment Models.pdf.txt...
string successfully retrieved.

['Abstract']
['1. Introduction']
['2. Related Work']
['3. Proposed Approach']
['5. Experiments']
['6. Conclusions']
['7. Acknowledgments']
['21845\nReferences']

Updating google sheet row 67
